In [ ]:

#Import Libraries
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from pickle import load
from tensordash.tensordash import Tensordash


In [ ]:
dataset = pd.read_csv('All-seasons.csv/data.csv')
characters = ['Stan','Kyle','Cartman']
dataset = dataset[dataset['Character'].isin(characters)]["Line"]
dataset.shape

In [ ]:
max_words = 50000 tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataset.values)
sequences = tokenizer.texts_to_sequences(dataset.values)
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)
print('Vocabulary size in this corpus: ', vocab_size)

In [ ]:
sentence_len = 20
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [ ]:
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(150, return_sequences=True),
    LSTM(150),
    Dense(150, activation='relu'),
    Dense(150, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
histories = Tensordash(ModelName = 'South Park Dialogue',
                       email = 'pranjal.kalbag@gmail.com')

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(np.asarray(trainX), pd.get_dummies(np.asarray(trainy)), batch_size=128, epochs=30, callbacks = [histories])

In [ ]:
input_string = input("Enter String:")
max_len = 20
tokenized_sent = tokenizer.texts_to_sequences([input_string])
max_len = max_len+len(tokenized_sent[0])
while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-19:],maxlen=19)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax()+1)
pred_string = " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))
print(pred_string)